This is the modified version of my submission for one of the assignments in UC Davis MSBA'24 - course 'Data Design & Representation'.

In [1]:
# libraries
import os
import re
import csv
import time
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import mysql.connector
import warnings

warnings.filterwarnings("ignore") # turn off warnings

## Web Scraping Indeed.com
1. Search target position ('data scientist' in this case)

In [2]:
# set up wendrivier
driver = webdriver.Chrome()
driver.implicitly_wait(10)
driver.set_script_timeout(120)
driver.set_page_load_timeout(10)

target_url = "https://www.indeed.com"

driver.get(target_url) # go to indeed.com
time.sleep(3)

# search for data scientist
input = driver.find_element(By.CSS_SELECTOR, '#text-input-what')
time.sleep(5) # 5s
input.send_keys('data scientist\n')
time.sleep(3) # 3s

2. Save the result to local drive

In [ ]:
# save the resulting HTML
page_html = driver.page_source #get page html

# locate the path to store the html result
save_location = os.getcwd() # get folder location
print('HTML Pages are saved here:',save_location)

# setup file name
file_name = 'job_datascientists.html'
file = os.path.join(save_location, file_name) 

# get page html
page_html = driver.page_source

# save file
with open(file, 'w+',encoding='utf-8') as f:
        f.write(page_html)
        f.close()
print("file saving completed!")        

time.sleep(3) # 3s

# close the window
driver.quit()

3. Organize the data

In [ ]:
# locate the path to store the html result
save_location = os.getcwd() # get folder location
print('HTML Pages are saved here:',save_location)

In [ ]:
# Loop through each file in the directory
for filename in os.listdir(save_location):
    # Check if the file ends with .html
    if filename.endswith(".html"):
        # Construct the full file path
        filepath = os.path.join(save_location, filename)
        #print(filename)
        # Read file to string
        with open(filepath, 'r', encoding='utf-8') as file:
            html = file.read()
            soup = BeautifulSoup(html, 'html.parser')
            
            # define a regular expression for tags contains 'Data Scientist'
            role_re = re.compile(r'Data .*')
            info_re = re.compile(r'jobMetaDataGroup.*')
            
            # search for related info
            titles = soup.find_all('a', attrs={"aria-label": role_re})
            companies = soup.find_all('span', attrs={"data-testid": 'company-name'})
            info = soup.find_all('div', attrs={"class": info_re})  

                
        # print results            
        for i in range(len(titles)):
            print('Job title: ' + titles[i].text)
            print('Company: ' + companies[i].text)
            #print(info[i].text)
            if 'Pay information not provided' in info[i].text:
                        print(info[i].text[:28])
            for item in info[i]:
                detail = item.find_all('div', attrs={"data-testid": "attribute_snippet_testid"})
                for i in range(len(detail)):
                    if detail[i] != None: # some info are not provided
                        print(detail[i].text)
                #details = None
            print()